# Question:

## What is the gross annual produced hydrocarbons on Norwegian Continental Shelf through time?

In [4]:
import numpy as np
import pandas as pd
import janitor

In [2]:
production_wellbores_monthly =  pd.read_csv("https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/field_production_gross_monthly&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=CSV&Top100=false&IpAddress=82.102.27.204&CultureCode=en")
production_wellbores_monthly.head()

,prfInformationCarrier,prfYear,prfMonth,prfPrdOilGrossMillSm3,prfPrdGasGrossBillSm3,prfPrdCondensateGrossMillSm3,prfPrdOeGrossMillSm3,prfPrdProducedWaterInFieldMillSm3,prfNpdidInformationCarrier
0,24/9-12 S (Frosk),2019,8,0.01705,0.00094,0.0,0.01799,0.00061,31140456
1,24/9-12 S (Frosk),2019,9,0.05557,0.00362,0.0,0.05919,0.00000,31140456
2,24/9-12 S (Frosk),2019,10,0.04403,0.00288,0.0,0.04691,0.00000,31140456
3,24/9-12 S (Frosk),2019,11,0.05350,0.00333,0.0,0.05682,0.00000,31140456
4,24/9-12 S (Frosk),2019,12,0.05825,0.00331,0.0,0.06156,0.00009,31140456


In [5]:
production_wellbores_monthly.columns = production_wellbores_monthly.columns.str.replace("prf", "")
production_wellbores_monthly = production_wellbores_monthly.clean_names(case_type="snake")

In [9]:
production_wellbores_monthly_LONG = (production_wellbores_monthly
    .drop(columns=["information_carrier", "npdid_information_carrier", "prd_oe_gross_mill_sm3"])
    .melt(
        id_vars=["year", "month"],
        var_name="hc_phase",
        value_name="prd_gross_mill_sm3"
        )
    .groupby(["year", "hc_phase"])
    .agg(prd_gross_mill_sm3_year = pd.NamedAgg(column="prd_gross_mill_sm3", aggfunc="sum"))
    .reset_index()
    ).sort_values(["year", "hc_phase"])

In [10]:
production_wellbores_monthly_LONG.head()

,year,hc_phase,prd_gross_mill_sm3_year
0,1971,prd_condensate_gross_mill_sm3,0.00000
1,1971,prd_gas_gross_bill_sm3,0.10294
2,1971,prd_oil_gross_mill_sm3,0.35712
3,1971,prd_produced_water_in_field_mill_sm3,0.00000
4,1972,prd_condensate_gross_mill_sm3,0.00000


In [11]:
import altair as alt
alt.data_transformers.disable_max_rows() # default is 5000 rows warning
#alt.renderers.enable('notebook')

domain = ['prd_condensate_gross_mill_sm3', 'prd_gas_gross_bill_sm3',
       'prd_oil_gross_mill_sm3', 'prd_produced_water_in_field_mill_sm3']
range_ = ['pink', 'red', 'green', 'blue']

annual_production_plot = (alt
    .Chart(production_wellbores_monthly_LONG)
    .mark_bar(opacity=0.3)
    .encode(
        x="year:Q",
        y="prd_gross_mill_sm3_year:Q",
        color=alt.Color('hc_phase', scale=alt.Scale(domain=domain, range=range_)),
        row="hc_phase:N"
        )
    .properties(height=100)
)

annual_production_plot

alt.Chart(...)

In [12]:
(annual_production_plot
    .encode(tooltip=['prd_gross_mill_sm3_year', 'year', 'hc_phase'])
    .interactive())

alt.Chart(...)